In [1]:
import requests
import os
import json
import time

Create env.txt file in directory. First line should be your github ID, second line should be authentication token.

In [2]:
Path = os.getcwd() + "\env.txt"
def getCredentials(path=Path):
    with open(path,"r") as file:
        contents = file.read()
        creds = contents.split("\n")
        user = creds[0]
        token = creds[1]
    return user,token
user,token = getCredentials()
url_headers = {
                "Accept" : "application/vnd.github.v3+json",
            }

filterMilestone(item) takes in a milestone object and returns an object with keys "title","state","description","due_on". These are used to create a milestone object. We have to filter to compare if milestones in source repo are different than milestones in destination repo.

In [3]:
def filterMilestone(item):
    your_keys = ["title","state","description","due_on"]
    milestone = { your_key: item[your_key] for your_key in your_keys }
    return milestone

In [4]:
def getMilestones(sourceRepo,org,Headers=url_headers,Auth=(user,token)):
    getMilestonesUrl = "https://api.github.com/repos/"+org+"/"+sourceRepo+"/milestones";
    time.sleep(1)
    response = requests.get(getMilestonesUrl, headers=Headers,auth=Auth)
    milestones = response.json()
    milestones = list(map(lambda item: filterMilestone(item),milestones))
    return milestones

# getMilestones("destination-repo","wsu-cpts489-fa21")

createMilestones(sourceRepo,repo,org,Headers = url_headers, Auth=(user,token)) clones milestones from source repo to repo. It checks what milestones already exist. Only new milestones are cloned.

In [5]:
def createMilestones(milestones,repo,org,Headers = url_headers, Auth=(user,token)):
    repo_milestone_url ="https://api.github.com/repos/"+org+"/"+repo+"/milestones"
    time.sleep(1)
    response = requests.get(repo_milestone_url, headers=Headers,auth=Auth)
    existing_milestones = response.json()
    existing_milestones = list(map(lambda item: filterMilestone(item),existing_milestones))
    print(f'{len(existing_milestones)} existing milestones found.')
    new_milestones = []
    for milestone in milestones:
        if not milestone in existing_milestones:
            new_milestones.append(milestone)
    count = 0
    for milestone in new_milestones:
        time.sleep(1)
        response = requests.post(repo_milestone_url,data=json.dumps(milestone),headers=Headers,auth=Auth)
        result = response.json()
        count += 1
    print(f'{count} new miletsones created.')

    

filterIssue(item) takes in an issue object and returns an object with keys "title","number","body","labels","milestone". These are used to create a new issue object. We have to filter to compare if issues in source repo are different than issues in destination repo.

In [6]:
def filterIssue(item):
    your_keys = ["id","title","number","body","labels","milestone"]
    issue = { your_key: item[your_key] for your_key in your_keys }
    if "milestone" in issue.keys() and issue["milestone"] != None :
        issue["milestone"] = issue["milestone"]["number"]
    else:
        issue["milestone"]=None
    return issue

In [7]:
def getIssues(sourceRepo,org,Headers=url_headers,Auth=(user,token)):
    get_issues_url  = "https://api.github.com/repos/"+org+"/"+sourceRepo+"/issues";
    time.sleep(1)
    response = requests.get(get_issues_url,params={"state":"open","per_page":100},headers=Headers, auth=Auth)
    source_issues =response.json()
    source_issues = list(map(lambda item: filterIssue(item),source_issues))
    return source_issues

# source_issues = getIssues("Team_Project_Starting_Repo","wsu-cpts489-fa21")


createIssues(sourceRepo,repo,org,Headers = url_headers, Auth=(user,token)) takes in sourceRepo, repo and clones issues from sourceRepo to repo. It checks what issue titles already exist in the repo. Only issues with different titles are cloned into destination repo.

In [8]:
def createIssues(source_issues,repo,org,Headers = url_headers, Auth=(user,token)):
    
    repo_issues_url ="https://api.github.com/repos/"+org+"/"+repo+"/issues"
    time.sleep(1)
    response = requests.get(repo_issues_url,params={"state":"all","per_page":100}, headers=Headers,auth=Auth)
    existing_issues = response.json()
    existing_issues = list(map(lambda item: filterIssue(item),existing_issues))
    print(f'{len(existing_issues)} existing issues found.')
    existing_issue_titles = [x["title"] for x in existing_issues]

    new_issues = []
    
    for issue in source_issues:
        title = issue["title"]
        if not title in existing_issue_titles:
            new_issues.append(issue)
    
    count =0
    created_issues = []
    for item in new_issues:
        item.pop("id")
        time.sleep(3)
        retry = True
        while(retry):
            response = requests.post(repo_issues_url,data=json.dumps(item),headers=Headers,auth=Auth)
            result = response.json()
            if not "title" in result.keys() and "message" in result.keys():
                print("request timed out....sleeping")
                time.sleep(10*60)
                retry = True
            else:
                print("Request processed - cloned issue# " + str(result["title"]))
                created_issues.append(result)
                retry = False

        count += 1
        
    print(f'{len(created_issues)} new issues created.')
    total_issues = existing_issues
    total_issues = total_issues +  list(map(lambda item: filterIssue(item),created_issues))
    print(f'{len(total_issues)} issues exist in the repo now.')
    return total_issues




In [9]:
def getProjectBoard(sourceRepo,project_name,org,Headers = url_headers, Auth=(user,token)):
    source_project_url = "https://api.github.com/repos/"+org+"/"+sourceRepo+"/projects"

    response = requests.get(source_project_url,headers=Headers,auth=Auth)
    projects = response.json()
    project = list(filter(lambda item: item["name"]==project_name,projects))
    if len(project)>0:
        project = project[0] 
        project_id = str(project["id"])
        project_columns_url = "https://api.github.com/projects/"+project_id+"/columns"

        response = requests.get(project_columns_url,headers=Headers,auth=Auth)
        columns = response.json()
        
        project_columns = []
        if len(columns)>0:
            for column in columns:
                item = {}
                

                column_id = str(column["id"])
                cards_url = "https://api.github.com/projects/columns/"+column_id+"/cards"

                response = requests.get(cards_url,params={"per_page":100},headers=Headers,auth=Auth)
                cards = response.json()
                column_issues = []
                if len(cards)>0:
                    for card in cards:

                        if "content_url" in card:

                            response = requests.get(card["content_url"],headers=Headers,auth=Auth)
                            issue_title = response.json()["title"]
                            column_issues.append(issue_title)

                name = column["name"]
                item[name]=column_issues
                project_columns.append(item)

        return project_columns
    else:
        project = None
        return None
        
# project_columns = getProjectBoard("Team_Project_Starting_Repo","CptS 489 Team Project","wsu-cpts489-fa21")
    

getProjectBoard(repo,org,Headers = url_headers, Auth=(user,token)) takes in the destination repo and creates a kanban board for it with columns "icebox", "to do", "in progress", "under review" and "done" . It checks if the kanban board already exists. A kanban board is created only if it does not exist. It returns todo_id, which is the id of the column "to do". This id is used to add new cards to the kanban board.

In [10]:
def createProjectBoard(repo,org,project_columns,projectBoard,Headers = url_headers, Auth=(user,token)):

    project_item = {"name": projectBoard,
               "body":"Project board for tracking progress."}
    repo_project_url = "https://api.github.com/repos/"+org+"/"+repo+"/projects"
   

    get_projects_url = "https://api.github.com/repos/"+org+"/"+repo+"/projects"
    time.sleep(1)
    response = requests.get(repo_project_url,headers=Headers,auth=Auth)
    projects = response.json()
    
    project = list(filter(lambda item: item["name"]==project_item["name"],projects))
    if len(project)>0:
        project = project[0] 
    else:
        project = None
   
    column_ids = {}
    if project!=None:
        print(f'projectboard |{project_item["name"]}| found!')
        project_id = str(project["id"])
        columns_url = "https://api.github.com/projects/"+project_id+"/columns"
        time.sleep(1)
        response = requests.get(columns_url,headers=Headers,auth=Auth)
        columns = response.json()
        for column in columns:
            name = column["name"]
            column_ids[name] = column["id"]

    
    else:
        print("creating project board.....")
        response = requests.post(repo_project_url,data=json.dumps(project_item), headers=Headers,auth=Auth)
        result = response.json()
        project_id = str(result["id"])
        column_url = "https://api.github.com/projects/"+project_id+"/columns"

        columns = [{"name":list(item.keys())[0]} for item in project_columns]
        
        print(columns)

        for column in columns:
            time.sleep(1)
            response = requests.post(column_url,data=json.dumps(column),headers=Headers,auth=Auth)
            result = response.json()
            name = column["name"]
            column_ids[name] = result["id"]

    
    return column_ids


# getProjectBoard("testRepo","wsu-cpts489-fa21",project_columns,"CptS 489 Team Project")
    
    
    

In [11]:
def issue_to_columns(project_columns):
    issue_columns = {}
    for column in project_columns:
        for issueList in column.values():
            for issue in issueList:
                issue_columns[issue] = list(column.keys())[0]
    return issue_columns

populateProjectBoard(repo,org,todo_id,Headers = url_headers, Auth=(user,token)) takes in the destination repo and todo_id (id of the todo column) and uses issues to create cards in the todo column of the project board. Only issues that are not in the todo column create new cards.

In [12]:
def populateProjectBoard(repo,org,project_columns,total_issues,
                         column_ids,projectBoard,Headers = url_headers, Auth=(user,token)):

    repo_project_columns = getProjectBoard(repo,projectBoard,org)
    
    source_issue_columns = issue_to_columns(project_columns)
    repo_issue_columns = issue_to_columns(repo_project_columns)

    print(f'{len(source_issue_columns)} issues exist on source kanban...')
    print(f'{len(repo_issue_columns)} issues exist on destination kanban...')
    
    count = 0
    cards = []
    for issue in total_issues:
        if issue["title"] in source_issue_columns.keys() and not issue["title"] in repo_issue_columns.keys():
            card = {"content_id":issue["id"],"content_type":"Issue"}
            column_id = column_ids[source_issue_columns[issue["title"]]]
            card_url = "https://api.github.com/projects/columns/"+str(column_id)+"/cards"
            time.sleep(3)
            retry = True
            while(retry):
                response = requests.post(card_url,data=json.dumps(card),headers=Headers,auth=Auth)
                result = response.json()
                if "message" in result.keys() and not "content_url" in result.keys():
                    print("request timed out....sleeping")
                    time.sleep(10*60)
                    retry = True
                else:
                    print("Request Processed - Card created!")
                    cards.append(result)
                    retry = False
    print(f"{len(cards)} new cards created.")

    
    

cloneRepos(sourceRepo, repoList, org) takes in sourceRepo: the repo to be copied, repoList: the list of repositories where sourceRepo will be cloned, org: the organisation. It clones milestones, issues and creates the project board for each repo in the repoList.

In [13]:
def cloneRepos(sourceRepo, repoList, org, projectBoard):
    print("------------------------------------------------------------------------------")
    milestones = getMilestones(sourceRepo,org)
    source_issues = getIssues(sourceRepo,org)
    project_columns = getProjectBoard(sourceRepo,projectBoard,org)

    for repo in repoList:
        print(f"Cloning items in {repo}....")
        createMilestones(milestones,repo,org)
        created_issues = createIssues(source_issues,repo,org)
        column_ids = createProjectBoard(repo,org,project_columns,projectBoard)
        populateProjectBoard(repo,org,project_columns,created_issues,column_ids,projectBoard)

Please set the following items in the next cell:
sourceRepo (the repo to be copied)
repoList (the list of repositories where source repo will be cloned)
org (the github organisation)
Run the cell to invoke cloneRepos.

In [14]:
repoList = ["testRepo"]
org = "wsu-cpts489-fa21"
sourceRepo = "destination-repo"
projectBoard = "CptS 489 Team Project"
cloneRepos(sourceRepo,repoList,org,projectBoard)



------------------------------------------------------------------------------
Cloning items in testRepo....
0 existing milestones found.
3 new miletsones created.
0 existing issues found.
Request processed - cloned issue# As a user, I can edit my account and profile settings so that I can customize my user experience
Request processed - cloned issue# As a user, I can log a round "live" by clicking buttons, so that I can capture detailed data on my round
Request processed - cloned issue# As a speedgolfer, I can edit my round data through the live round logging GUI so that I can correct mistakes
Request processed - cloned issue# As a user, I can view and edit my round data logged through live round logging UI after the fact, so that I can correct mistakes and undestand my rounds
Request processed - cloned issue# Testing #15: Live Round Logging GUI
Request processed - cloned issue# Testing #16: Editing of Data Logged through Live Round Logging UI
Request processed - cloned issue# Testing